In [2]:
!pip install streamlit pypdf langchain openai chromadb tiktoken huggingface_hub pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.9/271.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.5/415.5 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 55.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.2 MB/s eta 0:00:

In [ ]:
# this is if you don't want to use openAI embeddings so that you won't be charged
# from hugging_face mostly if you have a GPU
!pip install InstructorEmbedding sentence_transformers

In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.6 MB/s eta 0:00:00


In [1]:
%%writefile my_chat_app.py
import streamlit as st
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from htmlTemplates import css, user_template, bot_template
from PyPDF2 import PdfReader
from dotenv import load_dotenv, find_dotenv
import os
import openai


def get_pdf_document(files):
  #loader = PyPDFLoader(files)
  #pages = loader.load()
  text = ""
  for filex in files:
    pdf_reader = PdfReader(filex)
    for page in pdf_reader.pages:
      text += page.extract_text()
  return text

def split_and_get_chunks(documents,chunk_size=150, chunk_overlap=20):
  splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
  # splitter.split_documents
  # splitter.create_documents
  splitted_documents = splitter.split_text(documents)
  return splitted_documents


def create_vectore_stores(splitted_documents, directory="embeddings_vec/chroma/"):
  embeddings = OpenAIEmbeddings()
  PERSIST_DIRECTORY = directory
  # Chroma.from_documents
  vector_db = Chroma.from_texts(texts = splitted_documents,
                                    embedding=embeddings,
                                    persist_directory=PERSIST_DIRECTORY
                                    )
  return vector_db

def conversationQa(vector_store, search_type="mmr", k=4, fetch_k=20, chain_type = "stuff"):
  memory = ConversationBufferMemory(memory_key = "chat_history",
                                      return_messages = True)

  llm = ChatOpenAI(temperature=0)
  conversation_chain = ConversationalRetrievalChain.from_llm(
      llm,
      memory = memory,
      retriever = vector_store.as_retriever(search_type= search_type, search_kwargs = {"k":k, "fetch_k":fetch_k}),
      chain_type = chain_type
  )
  return conversation_chain

def question_and_answering(user_input):
  answer = st.session_state.conversations({"question": user_input})
  st.session_state.chat_history = answer["chat_history"]
  for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.write(user_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
        else:
            st.write(bot_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)


def main():
  _ = load_dotenv(find_dotenv())

  openai.api_key  = os.environ['OPENAI_API_KEY']
  st.set_page_config(page_title = "Chat WIth Your Data", page_icon=":books:")
  st.write(css, unsafe_allow_html = True)

  if "conversations" not in st.session_state:
    st.session_state.conversations= None
  if "chat_history" not in st.session_state:
    st.session_state.chat_history = None

  st.header("CHAT WITH YOUR DATA :books:")
  user_question = st.text_input("Ask a question about your documents")
  if user_question:
    question_and_answering(user_question)

  with st.sidebar:
    st.subheader("Your Documents")
    pdf_documents = st.file_uploader("upload your pdf files and click on 'Load' ", accept_multiple_files=True)
    if st.button("Load"):
      with st.spinner("loading"):
        # get pdf
        docs = get_pdf_document(pdf_documents)
        # createchunks
        chunks = split_and_get_chunks(docs)
        # vector store
        vectors = create_vectore_stores(chunks)
        # retrieve and chat
        st.session_state.conversations = conversationQa(vectors)


if __name__ == "__main__":
  main()

Writing my_chat_app.py


In [ ]:
!ngrok authtoken  os.environ[NGROK_AUTHTOKEN]

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2023-08-24 11:15:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  54.2MB/s    in 0.2s    

2023-08-24 11:15:37 (54.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://480b-34-148-102-204.ngrok-free.app


In [ ]:
!streamlit run /content/my_chat_app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.102.204:8501



In [ ]:
! pip install streamlit -q


In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.245.165.160


In [ ]:
! streamlit run my_chat_app.py & npx localtunnel --port 8501

[..................] | fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.165.160:8501

npx: installed 22 in 2.396s
your url is: https://light-guests-check.loca.lt
  Stopping...
^C
